In [ ]:
import cv2
import numpy as np
import time
from Quartz import CGWindowListCopyWindowInfo, kCGWindowListOptionOnScreenOnly, kCGWindowListExcludeDesktopElements, kCGNullWindowID
from mss import mss
import matplotlib.pyplot as plt
%matplotlib inline
import pytesseract
from AppKit import NSWindow, NSTitledWindowMask
import json
from difflib import get_close_matches
import re
import math
from IPython.display import clear_output
from adbutils import adb
import scrcpy.core as scrcpy
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QLabel
from PyQt5.QtGui import QImage, QPixmap
from PyQt5.QtCore import QTimer
import pandas as pd
import io

In [ ]:
phones = ['Pixel 3 XL', 'Pixel 7 Pro']
# Load the JSON files
with open('json_files/pk.json', 'r') as file:
    pokemon_names = json.load(file)

with open('json_files/great-league.json', 'r') as file:
    great_league = json.load(file)
with open('json_files/ultra-league.json', 'r') as file:
    ultra_league = json.load(file)
with open('json_files/master-league.json', 'r') as file:
    master_league = json.load(file)    
with open('json_files/moves.json', 'r') as file:
    moves = json.load(file)

alingment_info = """,1,2,3,4,5
1,,"1,2","2,3","3,4","4,5"
2,,,"1,3","1,2","2,5"
3,,"1,2",,"1,4","3,5"
4,,,"2,3",,"1,5"
5,,"1,2","1,4","3,4","""

df = pd.read_csv(io.StringIO(alingment_info), index_col=0)
def find_correct_first_three_counts(row, col):
    if pd.isna(df.at[row, col]):
        return None
    move_counts = [int(count) for count in df.at[row, col].split(',')]
    first_count, step = move_counts
    return [first_count + step * i for i in range(3)]

for index, row in df.iterrows():
    for col in df.columns:
        result = find_correct_first_three_counts(index, col)
        df.at[index, col] = result

df

In [ ]:
adb.connect("127.0.0.1:5037")
client = scrcpy.Client(device=adb.device_list()[0])
client.start(threaded=True)
print(client.device_name)
phone_t = phones.index(client.device_name)

In [ ]:
# Find the scrcpy window
def find_scrcpy_window(owner_name):
    window_info_list = CGWindowListCopyWindowInfo(kCGWindowListOptionOnScreenOnly | kCGWindowListExcludeDesktopElements, kCGNullWindowID)
    for window_info in window_info_list:
        window_owner_name = window_info.get('kCGWindowName', '')
        for i,owner in enumerate(owner_name):
            if owner in window_owner_name:
                print(f'Using phone:{owner}')
                return window_info,i
    return None,0

# Capture the scrcpy window
def capture_scrcpy_window(window, title_bar_height):
    x = window['kCGWindowBounds']['X']
    y = window['kCGWindowBounds']['Y']
    width = window['kCGWindowBounds']['Width']
    height = window['kCGWindowBounds']['Height']
    header_height = title_bar_height  # Height of the window header on macOS

    with mss() as sct:
        monitor = {'left': int(x), 'top': int(y) + header_height, 'width': int(width), 'height': int(height) - header_height}
        screenshot = sct.grab(monitor)
        screenshot_np = np.array(screenshot)

    return screenshot_np

def get_title_bar_height():
    window = NSWindow.alloc().initWithContentRect_styleMask_backing_defer_(
        ((0, 0), (100, 100)),
        NSTitledWindowMask,
        2,
        False
    )
    content_height = window.contentView().frame().size.height
    window_height = window.frame().size.height
    title_bar_height = window_height - content_height
    return title_bar_height

def crop_top(image, ratio):
    height, _, _ = image.shape
    cropped_height = int(height * ratio)
    return image[:cropped_height, :, :]

# Function to find the closest Pokémon name
def closest_pokemon_name(name, names_list):
    closest_name = get_close_matches(name, names_list, n=1, cutoff=0.6)
    if closest_name:
        return closest_name[0]
    return None

def calculate_move_counts(fast_move, charged_move):
    counts = []
    counts.append(math.ceil((charged_move['energy'] * 1) / fast_move['energyGain']))
    counts.append(math.ceil((charged_move['energy'] * 2) / fast_move['energyGain']) - counts[0])
    counts.append(math.ceil((charged_move['energy'] * 3) / fast_move['energyGain']) - counts[0] - counts[1])

    return counts

def turn_alignment(my_fast,opp_fast):
    pass

def get_moveset_and_counts(pokemon_name, pokemon_data, move_data):
    moveset = None
    for pokemon in pokemon_data:
        if pokemon_name.lower() == pokemon['speciesName'].lower():
            moveset = pokemon['moveset']
            break

    if moveset is None:
        return None

    fast_move = moveset[0]
    charged1 = moveset[1]
    charged2 = moveset[2]

    fast_move_data = None
    for move in move_data:
        if fast_move.lower() == move['moveId'].lower():
            fast_move_data = move
            break

    if fast_move_data is None:
        return None

    fast_count = round(fast_move_data['cooldown'] / 500)
    move_counts = {}
    move_counts[fast_move] = fast_count
    for charged_move_name in [charged1, charged2]:
        for move in move_data:
            if charged_move_name.lower() == move['moveId'].lower():
                move_counts[charged_move_name] = calculate_move_counts(fast_move_data, move)
                break

    return move_counts , fast_count

In [ ]:
scrcpy_window,phone = find_scrcpy_window(phones)
roi_adjust = [[23,175,415],[15,150,390]]
roi_adjust2 =[[50,370,860],[50,350,860]]
roi_adjust = roi_adjust[phone]
roi_adjust2 = roi_adjust2[phone_t]
roi_dim = [530,50]
title_bar_height = get_title_bar_height()
# Load the template images in color format
my_pokemon_template_color = cv2.imread('templates/my-temp2.png')
opp_pokemon_template_color = cv2.imread('templates/opp-temp2.png')
my_pokemon_template_color = cv2.resize(my_pokemon_template_color, (269, 77))
opp_pokemon_template_color = cv2.resize(opp_pokemon_template_color, (269, 77))
# Convert the template images to grayscale
my_pokemon_template = cv2.cvtColor(my_pokemon_template_color, cv2.COLOR_BGR2GRAY)
opp_pokemon_template = cv2.cvtColor(opp_pokemon_template_color, cv2.COLOR_BGR2GRAY)

In [ ]:
def update_ui():
    global opp_info_label, my_pokemon_label, opp_pokemon_label, my_moveset_label, opp_moveset_label, screenshot_label, correct_alignment

    screen = client.last_frame
    if screen is not None:
        my_roi = (roi_adjust2[0], roi_adjust2[1], roi_dim[0], roi_dim[1])
        opp_roi = (roi_adjust2[2], roi_adjust2[1], roi_dim[0], roi_dim[1])

        my_roi_img = screen[my_roi[1]:my_roi[1] + my_roi[3], my_roi[0]:my_roi[0] + my_roi[2]]
        opp_roi_img = screen[opp_roi[1]:opp_roi[1] + opp_roi[3], opp_roi[0]:opp_roi[0] + opp_roi[2]]

        gray_my_roi = cv2.cvtColor(my_roi_img, cv2.COLOR_BGR2GRAY)
        gray_opp_roi = cv2.cvtColor(opp_roi_img, cv2.COLOR_BGR2GRAY)

        # Apply Gaussian blur to the grayscale images
        blur_my_roi = cv2.GaussianBlur(gray_my_roi, (5, 5), 0)
        blur_opp_roi = cv2.GaussianBlur(gray_opp_roi, (5, 5), 0)

        # Apply binary thresholding
        _, thresh_my_roi = cv2.threshold(blur_my_roi, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        _, thresh_opp_roi = cv2.threshold(blur_opp_roi, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # Apply morphological operations to remove noise
        kernel = np.ones((2, 2), np.uint8)
        thresh_my_roi = cv2.morphologyEx(thresh_my_roi, cv2.MORPH_OPEN, kernel)
        thresh_opp_roi = cv2.morphologyEx(thresh_opp_roi, cv2.MORPH_OPEN, kernel)

        # Use pytesseract to extract text from the processed images
        my_info = pytesseract.image_to_string(thresh_my_roi, config='--psm 6').strip()
        opp_info = pytesseract.image_to_string(thresh_opp_roi, config='--psm 6').strip()

        # Display the extracted Pokémon name and CP value
        print("My Info:", my_info)
        print("Opponent Info:", opp_info)

        # Extract Pokémon names using regex
        my_info_match = re.search(r'[A-Z][a-z]+', my_info)
        opp_info_match = re.search(r'[A-Z][a-z]+', opp_info)
        # Find and correct the Pokémon names
        corrected_my_name = closest_pokemon_name(my_info, pokemon_names)
        corrected_opp_name = closest_pokemon_name(opp_info, pokemon_names)

        if my_info_match and opp_info_match:
            my_info_name = my_info_match.group(0)
            opp_info_name = opp_info_match.group(0)

            # Find and correct the Pokémon names
            corrected_my_name = closest_pokemon_name(my_info_name, pokemon_names)
            corrected_opp_name = closest_pokemon_name(opp_info_name, pokemon_names)

            if corrected_opp_name:
                opp_move_counts , opp_fast_move_turns = get_moveset_and_counts(corrected_opp_name, great_league, moves)
                opp_moveset_text = ', '.join([f"{move.lower()}: {count}" for move, count in opp_move_counts.items()])
                opp_moveset_label.setText(f"Opponent Moveset: {opp_moveset_text}")
                print(f"Opponent Pokémon: {corrected_opp_name}")
                print(opp_move_counts)
            else:
                print(f"Invalid Pokémon name: {opp_info_name}")

            if corrected_my_name:
                my_move_counts, my_fast_move_turns = get_moveset_and_counts(corrected_my_name, great_league, moves)
                my_moveset_text = ', '.join([f"{move.lower()}: {count}" for move, count in my_move_counts.items()])
                my_moveset_label.setText(f"My Moveset: {my_moveset_text}")
                print(f"My Pokémon: {corrected_my_name}")
                print(my_move_counts)
            else:
                print(f"Invalid Pokémon name: {my_info_name}")

        else:
            print("Could not extract Pokémon names from the OCR output.")

        # Draw rectangles around the ROI
        roi_color = (0, 255, 0)  
        screen_with_rois = cv2.rectangle(screen.copy(), (my_roi[0], my_roi[1]), (my_roi[0] + my_roi[2], my_roi[1] + my_roi[3]), roi_color, 2)
        screen_with_rois = cv2.rectangle(screen_with_rois, (opp_roi[0], opp_roi[1]), (opp_roi[0] + opp_roi[2], opp_roi[1] + opp_roi[3]), roi_color, 2)

        # Update the UI labels
        my_pokemon_label.setText(f"My Pokémon: {corrected_my_name}")
        opp_pokemon_label.setText(f"Opponent Pokémon: {corrected_opp_name}")

        if corrected_my_name is None:
            opp_moveset_label.setText(f"Opponent Moveset: {None}")
        if corrected_opp_name is None:
            my_moveset_label.setText(f"My Moveset: {None}")

        if corrected_my_name is not None and corrected_opp_name is not None:
            my_count = my_fast_move_turns
            opp_count = opp_fast_move_turns
            correct_count = df.loc[my_count,str(opp_count)]
            correct_alignment.setText(f"Correct Alignemnt: {correct_count}")

        # Resize the image and fix the colors
        scale_percent = 10
        width = int(screen_with_rois.shape[1] * scale_percent / 100)
        height = int(screen_with_rois.shape[0] * scale_percent / 100)
        dim = (width, height)
        resized_image = cv2.resize(screen_with_rois, dim, interpolation=cv2.INTER_AREA)
        resized_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB)

        # Update the screenshot display
        height, width, channel = resized_image.shape
        bytes_per_line = channel * width
        qimage = QImage(resized_image.data, width, height, bytes_per_line, QImage.Format_RGB888)
        pixmap = QPixmap.fromImage(qimage)
        screenshot_label.setPixmap(pixmap)
        clear_output(wait=True)
        # Schedule the next update
        QTimer.singleShot(1000, update_ui)


app = QApplication(sys.argv)
window = QWidget()
window.setWindowTitle("Pokémon Information Display")
layout = QVBoxLayout()

opp_pokemon_label = QLabel("Opponent Pokémon:")
layout.addWidget(opp_pokemon_label)

opp_moveset_label = QLabel()
layout.addWidget(opp_moveset_label)

correct_alignment = QLabel()
layout.addWidget(correct_alignment)

my_pokemon_label = QLabel("My Pokémon:")
layout.addWidget(my_pokemon_label)

my_moveset_label = QLabel()
layout.addWidget(my_moveset_label)

screenshot_label = QLabel()
layout.addWidget(screenshot_label)

window.setLayout(layout)
window.show()

# Start the update loop
QTimer.singleShot(1000, update_ui)

sys.exit(app.exec_())